In [41]:
import pandas as pd
import numpy as np

# Function to convert age (years/months) into weeks
def age_to_weeks(age):
    if 'year' in age:
        years = int(age.split()[0]) # Capture the numeric value before the "year" substring
        return years * 52 # Converts year to weeks (1 year == 52 weeks)
    elif 'month' in age:
        months = int(age.split()[0]) # Captures the numeric value before the "month" substring
        return months * 4 # Converts month to weeks (1 month == 4 weeks)
    elif 'week' in age:
        week = int(age.split()[0]) # Captures the numeric value before the "week" substring
        return week
    else:
        return 0 # If there is a value outside the expected parameters

# Color Mapping Dictionary
color_map = {
    'orange': ['orange', 'red', 'tan', 'buff', 'flame'],
    'black': ['black', 'seal', 'brown brindle', 'torbie', 'tortie', 'tricolor', 'calico'],
    'white': ['white', 'cream'],
    'brown': ['brown', 'chocolate', 'fawn'],
    'grey': ['gray', 'blue', 'silver'],
    'yellow': ['yellow'],
}

# Function to map a specific color to a primary color
def map_color(color):
    color = color.lower()  # Converts to lowercase for easier comparison
    for primary_color, color_list in color_map.items():
        for specific_color in color_list:
            if specific_color in color:
                return primary_color
    return color  # If the color is not in the mapping, we keep it as it is

# Function to treat multiple colors (color transition)
def handle_multiple_colors(color):
    # Separates colors that are with '/' or ' '
    if '/' in color:
        colors = color.split('/')
    elif ' ' in color:
        colors = color.split(' ')
    else:
        colors = [color]

    # Maps each color separately
    mapped_colors = [map_color(c) for c in colors]

    # Returns the combined colors (e.g. 'orange/white')
    return '/'.join(mapped_colors)

def match_duplicates(color):
    
    # Erases multiple spaces
    color = ' '.join(color.split())
    
    # Splits colors by "/"
    if '/' in color:
        colors = color.split('/')
    else:
        colors = color.split(' ')
    
    # Removes duplicates
    unique_colors = list(dict.fromkeys(colors))
    
    # Recombines non-duplicate colors in only one string, separated by "/"
    return '/'.join(unique_colors)
        

In [42]:
# Loads the data
df = pd.read_csv('Dataset/train.csv')

# View the data
display(df)


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,A683430,NaN,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,A667013,NaN,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
...,...,...,...,...,...,...,...,...,...,...
26724,A702446,NaN,2015-05-14 11:56:00,Transfer,Partner,Cat,Intact Male,1 month,Domestic Shorthair Mix,Brown Tabby/White
26725,A718934,NaN,2016-01-20 18:59:00,Transfer,SCRP,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Brown Tabby
26726,A698128,Zeus,2015-03-09 13:33:00,Adoption,NaN,Dog,Neutered Male,4 years,Old English Bulldog Mix,White/Tan
26727,A677478,NaN,2014-04-27 12:22:00,Transfer,Partner,Cat,Intact Male,4 weeks,Domestic Shorthair Mix,Black


In [43]:
# Data processing
df = df.drop(columns=['AnimalID','OutcomeSubtype']) #Remove useless columns

# Applies treatment to multiple colors in the 'Color' column
df['Mapped_Color'] = df['Color'].apply(handle_multiple_colors)

# Groups duplicated colors that are separated by "/" in only one color
df['Mapped_Color'] = df['Mapped_Color'].apply(match_duplicates)


# Part dedicated to group all the rare colors into the 'Other_Colors' category
color_counts = df['Mapped_Color'].value_counts()

# Defines a frequency threshold for rare colors ( currently given as 1% of the amount of examples in the dataset)
threshold = 267
rare_colors = color_counts[color_counts < threshold].index

# Substitutes the rare colors for the column 'Other_colors'
df['Mapped_Color'] = df['Mapped_Color'].apply(lambda x: 'Other_Colors' if x in rare_colors else x)

# Applies One-Hot Encoding to 'Mapped_Color'
mapped_color_dummies = pd.get_dummies(df['Mapped_Color'])

# Replaces the names of the animals by the characteristic of having a name or not
df['Name'] = df['Name'].apply(lambda x: 'yes' if pd.notna(x) and x != '' else 'no')

# Drops na values especifically in this point, since the most where names and OutcomeSubtypes (which are all treated)
df = df.dropna()

# Converts 'AgeuponOutcome' to weeks
df['AgeuponOutcomeWeeks'] = df['AgeuponOutcome'].apply(age_to_weeks) # Converting months and years to weeks

In [44]:
#Exploratory data analysis with statistics

# 0) Name column ratio analysis
outcome_proportion = df['Name'].value_counts(normalize=True)*100 # Normalize returns the proportion of each element == Num_ElementX/Num_total
print("Proporção de nomes (sim/não):")
print(outcome_proportion)

# 1) OutcomeType column ratio analysis
outcome_proportion = df['OutcomeType'].value_counts(normalize=True)*100 # Normalize returns the proportion of each element == Num_ElementX/Num_total
print("\nProporção de cada item do OutcomeType:")
print(outcome_proportion)

# 2) AnimalType column ratio analysis
animal_proportion = df['AnimalType'].value_counts(normalize=True)*100
print("\nProporção de cada item do AnimalType:")
print(animal_proportion)

# 3) Proportion analysis of the Breed column (Mestiza X Pure)
# Creation of a subtable for mixed or pure breed classification and label-encoding
df['Breed'] = df['Breed'].apply(lambda x: 0 if 'Mix' in x else 1)  # lambda x is a temporary function that takes as input the parameter x (element of the breed column)
df['BreedType'] = df['Breed']
df = df.drop(columns=['Breed'])
breed_proportion = df['BreedType'].value_counts(normalize=True)*100
print("\nProporção de BreedType (Mestiça/Pura)")
print(breed_proportion)

# 4) AgeuponOutcome column ratio analysis# Create a column that indicates which range each 'Age_in_Weeks' value falls into
df['Intervalo'] = pd.cut(df['AgeuponOutcomeWeeks'], bins=bins, right=False)
frequencia_absoluta = df['Intervalo'].value_counts().sort_index() # Calculate absolute frequency (number of observations in each interval)
frequencia_relativa = frequencia_absoluta / len(df) * 100 # Calculate relative frequency (proportion of observations in each interval)

# Create a table with absolute and relative frequencies
tabela_frequencia = pd.DataFrame({
    'Frequência Absoluta': frequencia_absoluta,
    'Frequência Relativa (%)': frequencia_relativa
})

print("\nProporção de AgeuponOutcome (em semanas)")
print(tabela_frequencia)

# 5) SexuponOutcome column ratio analysis
# A) Proportion of castrated and non-castrated
castrated_proportion = df['SexuponOutcome'].apply(lambda x: 'Castrado' if 'Spayed' in x or 'Neutered' in x else 'Não Castrado').value_counts(normalize=True) * 100
print("\nProporção Castrados e Não Castrados:")
print(castrated_proportion)

# B) Proportion of Females and Males
gender_proportion = df['SexuponOutcome'].apply(lambda x: 'Macho' if 'Male' in x else 'Fêmea').value_counts(normalize=True) * 100
print("\nProporção de Machos e Fêmeas:")
print(gender_proportion)

# C) Proportion of females and males castrated and not castrated
gender_castrated_proportion = df['SexuponOutcome'].apply(lambda x:
    'Macho Castrado' if 'Neutered Male' in x else
    'Macho Não Castrado' if 'Intact Male' in x else
    'Fêmea Castrada' if 'Spayed Female' in x else
    'Fêmea Não Castrada').value_counts(normalize=True) * 100
print("\nProporção de Machos e Fêmeas (Castrados e Não Castrados):")
print(gender_castrated_proportion)

# Filter only Females (Spayed Female and Intact Female)
females_only = df['SexuponOutcome'].apply(lambda x: 'Fêmea Castrada' if 'Spayed Female' in x else 'Fêmea Não Castrada' if 'Intact Female' in x else None)

# Remove None values ​​(if they are males or unknown values)
females_only = females_only.dropna()

# Calculate the proportion of castrated and uncastrated females
gender_castrated_proportion = females_only.value_counts(normalize=True) * 100
print("\nProporção de Fêmeas (Castradas e Não Castradas):")
print(gender_castrated_proportion)

# Filter only Males (Neutered Male and Intact Male)
males_only = df['SexuponOutcome'].apply(lambda x: 'Macho Castrado' if 'Neutered Male' in x else 'Macho Não Castrado' if 'Intact Male' in x else None)

# Remove None values ​​(if they are females or unknown values)
males_only = males_only.dropna()

# Calculate the proportion of castrated and uncastrated males
gender_castrated_proportion = males_only.value_counts(normalize=True) * 100
print("\nProporção de Machos (Castrados e Não Castrados):")
print(gender_castrated_proportion)

# 6) Color column proportion analysis
color_proportion = df['Color'].value_counts(normalize=True)*100
print("\nProporção de Color:")
print(len(color_proportion))

color_proportion = df['Mapped_Color'].value_counts(normalize=True)*100
print("\nProporção de Mapped_Color:")
print(len(color_proportion))

for row in range(len(color_proportion)):
  indice = color_proportion.index[row]
  valor = color_proportion.iloc[row]
  print(f'Índice: {indice}, Valor: {valor}')

  

Proporção de nomes (sim/não):
Name
yes    71.272931
no     28.727069
Name: proportion, dtype: float64

Proporção de cada item do OutcomeType:
OutcomeType
Adoption           40.318233
Transfer           35.215275
Return_to_owner    17.914639
Euthanasia          5.814302
Died                0.737551
Name: proportion, dtype: float64

Proporção de cada item do AnimalType:
AnimalType
Dog    58.378884
Cat    41.621116
Name: proportion, dtype: float64

Proporção de BreedType (Mestiça/Pura)
BreedType
0    83.414452
1    16.585548
Name: proportion, dtype: float64

Proporção de AgeuponOutcome (em semanas)
             Frequência Absoluta  Frequência Relativa (%)
Intervalo                                                
[0, 52)                    11890                44.515163
[52, 104)                   3969                14.859603
[104, 156)                  3742                14.009734
[156, 208)                  1823                 6.825159
[208, 260)                  1071                 

In [45]:
# Categoric to numerical data conversion

# Drops 'Mapped_Color' to leave just the other color columns
df = df.drop(columns=['Mapped_Color'])

# Defines discrete intervals (bins) every 52 weeks, from 0 to 1040
bins = np.arange(0, 1041, 52)  # This creates 52 week intervals (which is around 1 year)

# Drops unprocessed columns
df = df.drop(columns=['Color', 'DateTime', 'AgeuponOutcome'])

# Concatenates the new One-Hot Encoding columns to the DataFrame
df = pd.concat([df, mapped_color_dummies], axis=1)

# Imports label encoding utilities
from sklearn.preprocessing import LabelEncoder

# Applies Label Encoding to 'Name'
label_encoder_name = LabelEncoder()
df['Name'] = label_encoder_name.fit_transform(df['Name'])

# Applies Label Encoding to 'AnimalType'
label_encoder_animal = LabelEncoder()
df['AnimalType'] = label_encoder_animal.fit_transform(df['AnimalType'])

# Applies Label Encoding to 'OutcomeType'
label_encoder_outcome = LabelEncoder()
df['OutcomeType'] = label_encoder_outcome.fit_transform(df['OutcomeType'])

# Applies One-Hot Encoding to 'SexuponOutcome'
df = pd.get_dummies(df, columns=['SexuponOutcome'], prefix='sex')

df = df.drop(columns=['Intervalo'])

# Showa the processed table with most numeric values

print('Dataset modificado')
display(df)



Dataset modificado


,Name,OutcomeType,AnimalType,AgeuponOutcomeWeeks,BreedType,Other_Colors,black,black/brown,black/orange,black/white,...,orange/white,white,white/black,white/brown,white/orange,sex_Intact Female,sex_Intact Male,sex_Neutered Male,sex_Spayed Female,sex_Unknown
0,1,3,1,52.0,0.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,1,2,0,52.0,0.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,1,0,1,104.0,0.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,0,4,0,3.0,0.0,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,0,4,1,104.0,1.0,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22380,2,5,2,NaN,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22448,2,5,2,NaN,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
24328,2,5,2,NaN,NaN,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26551,2,5,2,NaN,NaN,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
